In [1]:
import tensorflow as tf
import tensorflow_quantum as tfq
import cirq
import sympy
import cmath
import numpy as np
from scipy import linalg
import sys
import time

# visualization tools
%matplotlib inline
import matplotlib.pyplot as plt
from cirq.contrib.svg import SVGCircuit
from IPython import display

In [2]:
qubits = cirq.GridQubit.rect(1, 4)

In [24]:
def generate_even_block(block_number):
    """ Function for generating the even blocks
        block_number: Block number, it has to be even.
        return: ciq.Circuit """
    params = sympy.symbols(['theta_{}'.format(n) for n in range((block_number-1)*4,block_number*4)])

    # create the parameterized circuit
    circuit = cirq.Circuit(
        cirq.rz(params[0])(qubits[0]),
        cirq.rz(params[1])(qubits[1]),
        cirq.rz(params[2])(qubits[2]),
        cirq.rz(params[3])(qubits[3]),
        cirq.CZ(qubits[0],qubits[1]),
        cirq.CZ(qubits[0],qubits[2]),
        cirq.CZ(qubits[0],qubits[3]),
        cirq.CZ(qubits[1],qubits[2]),
        cirq.CZ(qubits[1],qubits[3]),
        cirq.CZ(qubits[2],qubits[3])
    )
    
    return circuit

In [25]:
def generate_odd_block(block_number):
    """ Function for generating the odd blocks
        block_number: Block number, it has to be odd.
        return: ciq.Circuit """
    params = sympy.symbols(['theta_{}'.format(n) for n in range((block_number-1)*4,block_number*4)])

    # create the parameterized circuit
    circuit = cirq.Circuit(
        cirq.rx(params[0])(qubits[0]),
        cirq.rx(params[1])(qubits[1]),
        cirq.rx(params[2])(qubits[2]),
        cirq.rx(params[3])(qubits[3])
    )
    
    return circuit

In [28]:
c2 = generate_odd_block(1)

In [29]:
print(c2)

(0, 0): ───Rx(theta_0)───

(0, 1): ───Rx(theta_1)───

(0, 2): ───Rx(theta_2)───

(0, 3): ───Rx(theta_3)───


In [30]:
c2 += generate_even_block(2)

In [31]:
print(c2)

                                               ┌──┐
(0, 0): ───Rx(theta_0)───Rz(theta_4)───@───@────@─────────────
                                       │   │    │
(0, 1): ───Rx(theta_1)───Rz(theta_5)───@───┼────┼@────@───────
                                           │    ││    │
(0, 2): ───Rx(theta_2)───Rz(theta_6)───────@────┼@────┼───@───
                                                │     │   │
(0, 3): ───Rx(theta_3)───Rz(theta_7)────────────@─────@───@───
                                               └──┘


In [32]:
def generate_qnn(l):
    """ Function for generating qnn, containing l number of layers.
        l: number of layers, each layer contains one odd and one even block.
        return: ciq.Circuit """
    circuit = cirq.Circuit()
    for i in range(1,2*l+1):
        if i % 2 == 1:
            circuit += generate_odd_block(i)
        else:
            circuit += generate_even_block(i)
        
    return circuit

In [37]:
l=1
c = generate_qnn(l)
print(c)

                                               ┌──┐
(0, 0): ───Rx(theta_0)───Rz(theta_4)───@───@────@─────────────
                                       │   │    │
(0, 1): ───Rx(theta_1)───Rz(theta_5)───@───┼────┼@────@───────
                                           │    ││    │
(0, 2): ───Rx(theta_2)───Rz(theta_6)───────@────┼@────┼───@───
                                                │     │   │
(0, 3): ───Rx(theta_3)───Rz(theta_7)────────────@─────@───@───
                                               └──┘


In [96]:
params = tf.zeros([1,8*l ], tf.float32)
print(params)
params_names = sympy.symbols(['theta_{}'.format(n) for n in range(8*l)])
print(params_name)

tf.Tensor([[0. 0. 0. 0. 0. 0. 0. 0.]], shape=(1, 8), dtype=float32)
[theta_0, theta_1, theta_2, theta_3, theta_4, theta_5, theta_6, theta_7]


In [97]:
state_layer = tfq.layers.State()
state = state_layer(c, symbol_names=params_names, symbol_values=params)
print(state)

<tf.RaggedTensor [[(1+0j), 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j]]>


In [104]:
random_circuit = cirq.testing.random_circuit(qubits = qubits,n_moments = np.random.randint(low=1,high=5),\
                                                                op_density = 0.99999999)
print(random_circuit)
random_circuit = tfq.layers.State()(random_circuit)
print(random_circuit)

(0, 0): ───×───T───Y───iSwap───
           │           │
(0, 1): ───×───────@───iSwap───
                   │
(0, 2): ───Y───Y───X───×───────
                       │
(0, 3): ───────Z───────×───────
<tf.RaggedTensor [[(3.82137093032941e-15-4.371138828673793e-08j), (3.3407486883511458e-22-3.82137093032941e-15j), (-1.6703743441755729e-22-7.301438151149521e-30j), 0j, (-1-1.3113415775478643e-07j), (-8.742277657347586e-08-1.1464113214504704e-14j), (-3.3407486883511458e-22+3.82137093032941e-15j), 0j, (5.732056183735878e-15-4.371138828673793e-08j), (5.011123158744463e-22-3.82137093032941e-15j), (-1.6703743441755729e-22-1.4602876302299043e-29j), 0j, 0j, 0j, 0j, 0j]]>


In [ ]:
diff = state[0] - random_circuit[0]
conjugate_traspose_state = tf.transpose(diff,conjugate = True)
print()